# Cas Kaggle

**Nerea de La Torre Veguillas 1669013**

**Helena Boneu Vidal 1673129**

Hem utilitzat el dataset **Harry Potter Dataset**.

In [1762]:
import pandas as pd
import numpy as np

Carreguem el **dataset**:

Quan vam observar el datset per primer cop, vem veure que hi havien bastantes característiques dels personatges que no hi eren, per això vem decidir fer una cerca a internet i buscar la informació faltat. Tot i així, no vem poder aconseguir trobar totes les dades necessàries.

In [1763]:
df = 'C:/Users/nerea/OneDrive/Escritorio/cosas uni/3r/AP/kaggle/Characters.csv'

df = pd.read_csv(df, delimiter=';')
df.head(5)

,Id,Name,Gender,Job,House,Wand,Patronus,Species,Blood status,Hair colour,Eye colour,Loyalty,Skills,Birth,Death
0,1,Harry James Potter,Male,Student,Gryffindor,"11"" Holly phoenix feather",Stag,Human,Half-blood,Black,Bright green,Albus Dumbledore | Dumbledore's Army | Order o...,Parseltongue| Defence Against the Dark Arts | ...,31 July 1980,NaN
1,2,Ronald Bilius Weasley,Male,Student,Gryffindor,"12"" Ash unicorn tail hair",Jack Russell terrier,Human,Pure-blood,Red,Blue,Dumbledore's Army | Order of the Phoenix | Hog...,Wizard chess | Quidditch goalkeeping,1 March 1980,NaN
2,3,Hermione Jean Granger,Female,Student,Gryffindor,"10¾"" vine wood dragon heartstring",Otter,Human,Muggle-born,Brown,Brown,Dumbledore's Army | Order of the Phoenix | Hog...,Almost everything,"19 September, 1979",NaN
3,4,Albus Percival Wulfric Brian Dumbledore,Male,Headmaster,Gryffindor,"15"" Elder Thestral tail hair core",Phoenix,Human,Half-blood,Silver| formerly auburn,Blue,Dumbledore's Army | Order of the Phoenix | Hog...,Considered by many to be one of the most power...,Late August 1881,"30 June, 1997"
4,5,Rubeus Hagrid,Male,Keeper of Keys and Grounds | Professor of Care...,Gryffindor,"16"" Oak unknown core",NaN,Half-Human/Half-Giant,Part-Human (Half-giant),Black,Black,Albus Dumbledore | Order of the Phoenix | Hogw...,Resistant to stunning spells| above average st...,6 December 1928,NaN


In [1764]:
print("Columnes amb nans:\n")
nan_columns = df.columns[df.isna().sum()>0].tolist()
print (df.loc[:,nan_columns].isna().sum()/len(df)*100)

Columnes amb nans:

Job              5.000000
House           16.428571
Patronus         5.000000
Blood status     4.285714
Hair colour      0.714286
Eye colour      12.142857
Skills           5.000000
Birth            9.285714
Death           70.000000
dtype: float64


Eliminem les files dels **personatges** que **no** tenen **casa**.

In [1765]:
df = df.dropna(subset=['House'])

També veiem que les característiques 'Birth, Death' no són rellevants per a la nostra predicció. Per tant, les podriem eliminar del nostre dataset.

In [1766]:
df = df.drop(columns=['Birth', 'Death'])
print(df.keys())

Index(['Id', 'Name', 'Gender', 'Job', 'House', 'Wand', 'Patronus', 'Species',
       'Blood status', 'Hair colour', 'Eye colour', 'Loyalty', 'Skills'],
      dtype='object')


Mirem les **columnes** que tenen **nans**.

In [1767]:
print("Columnes amb nans:\n")
nan_columns = df.columns[df.isna().sum()>0].tolist()
print (df.loc[:,nan_columns].isna().sum()/len(df)*100)

Columnes amb nans:

Job              4.273504
Patronus         5.128205
Blood status     1.709402
Eye colour      10.256410
Skills           5.128205
dtype: float64


Haurem de fer **encoding** per les variables categòriques.
* Eye colour
* Hair colour
* Gender

In [1768]:
from sklearn.preprocessing import LabelEncoder

numeric_features = df.select_dtypes(np.number).keys()
non_numeric = ['Eye colour', 'Hair colour', 'Gender']
df.loc[:,non_numeric] = df.loc[:,non_numeric].apply(LabelEncoder().fit_transform)
df.head()

,Id,Name,Gender,Job,House,Wand,Patronus,Species,Blood status,Hair colour,Eye colour,Loyalty,Skills
0,1,Harry James Potter,1,Student,Gryffindor,"11"" Holly phoenix feather",Stag,Human,Half-blood,2,4,Albus Dumbledore | Dumbledore's Army | Order o...,Parseltongue| Defence Against the Dark Arts | ...
1,2,Ronald Bilius Weasley,1,Student,Gryffindor,"12"" Ash unicorn tail hair",Jack Russell terrier,Human,Pure-blood,16,1,Dumbledore's Army | Order of the Phoenix | Hog...,Wizard chess | Quidditch goalkeeping
2,3,Hermione Jean Granger,0,Student,Gryffindor,"10¾"" vine wood dragon heartstring",Otter,Human,Muggle-born,5,5,Dumbledore's Army | Order of the Phoenix | Hog...,Almost everything
3,4,Albus Percival Wulfric Brian Dumbledore,1,Headmaster,Gryffindor,"15"" Elder Thestral tail hair core",Phoenix,Human,Half-blood,22,1,Dumbledore's Army | Order of the Phoenix | Hog...,Considered by many to be one of the most power...
4,5,Rubeus Hagrid,1,Keeper of Keys and Grounds | Professor of Care...,Gryffindor,"16"" Oak unknown core",NaN,Half-Human/Half-Giant,Part-Human (Half-giant),2,0,Albus Dumbledore | Order of the Phoenix | Hogw...,Resistant to stunning spells| above average st...


Apliquem un KKN per trobar la característica **Eye colour** dels personatges que no la tenen.

In [1769]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
df['Eye colour'] = imputer.fit_transform(df[['Gender', 'Hair colour']])

A la categoria Species, hem borrat els parèntesis explicatius que hi ha a alguns personatges que són Human.

In [1770]:
# Eliminem espai del final
df['Species'] = df['Species'].str.strip()

# Eliminem els parèntesis explicatius de la columna 'Species'
df['Species'] = df['Species'].str.replace(r'\s*\(.*?\)', '', regex=True)

# Verifiquem els resultats
df['Species'].unique()

array(['Human', 'Half-Human/Half-Giant', 'Werewolf', 'Ghost'],
      dtype=object)

A la característica **Blood status** també apliquem un KKN per trobar les que ens falten.

In [1771]:
numeric_features = df.select_dtypes(np.number).keys()
non_numeric = ['Species']
df.loc[:,non_numeric] = df.loc[:,non_numeric].apply(LabelEncoder().fit_transform)

imputer = KNNImputer(n_neighbors=5)
df['Blood status'] = imputer.fit_transform(df[['Species']])

In [1772]:
print("Columnes amb nans:\n")
nan_columns = df.columns[df.isna().sum()>0].tolist()
print (df.loc[:,nan_columns].isna().sum()/len(df)*100)

nan_columns = df.columns[df.isna().sum()>0].tolist() # columnes que falta 1 o més valors al datset
print('\nNÚMERO DE NANS PER COLUMNA:')
print (df.loc[:,nan_columns].isna().sum()) # número de nans per columna

Columnes amb nans:

Job         4.273504
Patronus    5.128205
Skills      5.128205
dtype: float64

NÚMERO DE NANS PER COLUMNA:
Job         5
Patronus    6
Skills      6
dtype: int64


Característica **Job**

In [1773]:
# Eliminem espai del final
df['Job'] = df['Job'].str.strip()

# Eliminem els parèntesis explicatius de la columna 'Species'
df['Job'] = df['Job'].str.replace(r'\s*\(.*?\)', '', regex=True)

# Verifiquem els resultats
df['Job'].unique()

# Primer, ignorem les files amb NaNs a la columna 'Job' al contar les feines
df['Job_Count'] = df['Job'].apply(lambda x: len(x.split('|')) if pd.notna(x) else 0)

# Filtrem les files on 'Job_Count' es més gran que 1
multiple_jobs = df[df['Job_Count'] > 1]

# Imprimim els personatges que tenen més d'una feina
print("Personatges amb més d'una feina:")
print(multiple_jobs[['Job', 'Job_Count']])

print("\nEl màxim de feines que te algún dels personatges és de",max(multiple_jobs['Job_Count']))

#df.head()

Personatges amb més d'una feina:
                                                  Job  Job_Count
4   Keeper of Keys and Grounds | Professor of Care...          2
16  Student|Prefect|  Personal assistant to Barty ...          3
29  Professor of Transfiguration | Head of Gryffindor          2
36            Professor of Charms | Head of Ravenclaw          2
45           Professor of Potions | Head of Slytherin          2
50  Professor of Defence Against the Dark Arts | D...          2
81  Professor of Herbology | Head of Hufflepuff House          2
94                        Auror  | Minister for Magic          2
98           Head of Auror Office| Minister for Magic          2

El màxim de feines que te algún dels personatges és de 3


Com que hem vist que el màxim de feines és de 3, crearem 3 columnes addicionals separant les feines de cada personatge. Les columnes que crearem seran:
* Job_1 (per la 1ra feina)
* Job_2 (per la 2na feina)
* Job_3 (per la 3ra feina)

In [1774]:
# Separem les feines en 3 columnes
feines_separades = df['Job'].apply(lambda x: x.split('|') if pd.notna(x) else []).apply(pd.Series)

# Canviem el nom de les columnes
feines_separades.columns = [f'Job_{i+1}' for i in range(feines_separades.shape[1])]

# Afegim les columnes al df
df = pd.concat([df, feines_separades], axis=1)

df.head()

,Id,Name,Gender,Job,House,Wand,Patronus,Species,Blood status,Hair colour,Eye colour,Loyalty,Skills,Job_Count,Job_1,Job_2,Job_3
0,1,Harry James Potter,1,Student,Gryffindor,"11"" Holly phoenix feather",Stag,2,2.0,2,1.0,Albus Dumbledore | Dumbledore's Army | Order o...,Parseltongue| Defence Against the Dark Arts | ...,1,Student,NaN,NaN
1,2,Ronald Bilius Weasley,1,Student,Gryffindor,"12"" Ash unicorn tail hair",Jack Russell terrier,2,2.0,16,1.0,Dumbledore's Army | Order of the Phoenix | Hog...,Wizard chess | Quidditch goalkeeping,1,Student,NaN,NaN
2,3,Hermione Jean Granger,0,Student,Gryffindor,"10¾"" vine wood dragon heartstring",Otter,2,2.0,5,0.0,Dumbledore's Army | Order of the Phoenix | Hog...,Almost everything,1,Student,NaN,NaN
3,4,Albus Percival Wulfric Brian Dumbledore,1,Headmaster,Gryffindor,"15"" Elder Thestral tail hair core",Phoenix,2,2.0,22,1.0,Dumbledore's Army | Order of the Phoenix | Hog...,Considered by many to be one of the most power...,1,Headmaster,NaN,NaN
4,5,Rubeus Hagrid,1,Keeper of Keys and Grounds | Professor of Care...,Gryffindor,"16"" Oak unknown core",NaN,1,1.0,2,1.0,Albus Dumbledore | Order of the Phoenix | Hogw...,Resistant to stunning spells| above average st...,2,Keeper of Keys and Grounds,Professor of Care of Magical Creatures,NaN


A continuació, agruparem les feines de cada personatge en feines més globals. A més a més, els personatges que tinguin més d'una feina, triarem per a aquests la que és repeteix més vegades d'entre els personatges que tenen només una feina.

In [1775]:
# Creem un diccionari de mapeig per agrupar feines similars
feines_similars = {
    'Professor': ['Professor', 'Defence Against the Dark Arts'],
    'House Ghost': ['House Ghost'],
    'Founder': ['Founder'],
    'School Head' : ['Headmaster', 'Headmistress'],
    'Gringotts' : ['Gringotts'],
    'Hogwarts service' : ['Matron', 'Flying Instructor', 'Astronomer', 'keeper of keys'],
    'Magical Government' : ['Magical', 'Muggle Artefacts'],
    'Death eater' : ['unspeakable', 'voldemort', 'executioner'],
    'Barman' : ['barman'],   
    'Quidditch' : ['quidditch'],
    'Head of House' : ['head'],
    'Personal Assistant' : ['Personal assistant']              
}

def agrupar_feina(feina):
    # Ens assegurem que les paraules estiguin en minuscules i sense espais addicionals
    feina = feina.strip().lower()
    
    for categoria, similares in feines_similars.items():
        # Comparem la feina amb les paraules de la llista
        llista = [item.strip().lower() for item in similares]
        paraules_dividides = []
        for paraula in llista:
            paraula_separada = paraula.split(' ')
            paraules_dividides.append(' '.join(paraula_separada))
        for i in paraules_dividides:
             if i in feina:
                return categoria.lower()
    return feina

# Filtrem els personatges que tenen 1, 2 o 3 feines
df_job_1 = df[df['Job_Count'] == 1]
df_job_2 = df[df['Job_Count'] == 2]
df_job_3 = df[df['Job_Count'] == 3]

# Ens guardem també els personatges que no tenen cap feina
df_sense_job = df[df['Job_Count'] == 0]

# Apliquem la funció per agrupar les feines més similars de la columna 'Job_1'
df_job_1['Job_1_Agrupat'] = df_job_1['Job_1'].apply(agrupar_feina)

# Apliquem la funció per agrupar les feines més similars de la columna 'Job_2'
df_job_2['Job_1_Agrupat'] = df_job_2['Job_1'].apply(agrupar_feina)
df_job_2['Job_2_Agrupat'] = df_job_2['Job_2'].apply(agrupar_feina)

# Apliquem la funció per agrupar les feines més similars de la columna 'Job_3'
df_job_3['Job_1_Agrupat'] = df_job_3['Job_1'].apply(agrupar_feina)
df_job_3['Job_2_Agrupat'] = df_job_3['Job_2'].apply(agrupar_feina)
df_job_3['Job_3_Agrupat'] = df_job_3['Job_3'].apply(agrupar_feina)

# Contem quantes vegades és repeteixen les feines de la columna 'Job_1'
feines_repetides_job_1 = df_job_1['Job_1_Agrupat'].value_counts()

# Imprimim les feines i quantes vegeades és repeteixen
print("Feines repetides Job_1:\n")
print(feines_repetides_job_1)

# Concatenem tots els datasets
df_tot = pd.concat([df_sense_job, df_job_1, df_job_2, df_job_3], ignore_index=True, sort=False)

# Ordenem el dataset respecte Id
df_tot = df_tot.sort_values(by='Id')

df_tot.head()

Feines repetides Job_1:

Job_1_Agrupat
student               53
professor              9
auror                  7
death eater            7
house ghost            4
founder                4
magical government     4
hogwarts service       3
housewife              2
school head            2
gringotts              2
wandmaker              1
school governor        1
dragonologist          1
minister for magic     1
barman                 1
quidditch              1
Name: count, dtype: int64


C:\Users\nerea\AppData\Local\Temp\ipykernel_14672\531071686.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_job_1['Job_1_Agrupat'] = df_job_1['Job_1'].apply(agrupar_feina)
C:\Users\nerea\AppData\Local\Temp\ipykernel_14672\531071686.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_job_2['Job_1_Agrupat'] = df_job_2['Job_1'].apply(agrupar_feina)
C:\Users\nerea\AppData\Local\Temp\ipykernel_14672\531071686.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

,Id,Name,Gender,Job,House,Wand,Patronus,Species,Blood status,Hair colour,Eye colour,Loyalty,Skills,Job_Count,Job_1,Job_2,Job_3,Job_1_Agrupat,Job_2_Agrupat,Job_3_Agrupat
5,1,Harry James Potter,1,Student,Gryffindor,"11"" Holly phoenix feather",Stag,2,2.0,2,1.0,Albus Dumbledore | Dumbledore's Army | Order o...,Parseltongue| Defence Against the Dark Arts | ...,1,Student,NaN,NaN,student,NaN,NaN
6,2,Ronald Bilius Weasley,1,Student,Gryffindor,"12"" Ash unicorn tail hair",Jack Russell terrier,2,2.0,16,1.0,Dumbledore's Army | Order of the Phoenix | Hog...,Wizard chess | Quidditch goalkeeping,1,Student,NaN,NaN,student,NaN,NaN
7,3,Hermione Jean Granger,0,Student,Gryffindor,"10¾"" vine wood dragon heartstring",Otter,2,2.0,5,0.0,Dumbledore's Army | Order of the Phoenix | Hog...,Almost everything,1,Student,NaN,NaN,student,NaN,NaN
8,4,Albus Percival Wulfric Brian Dumbledore,1,Headmaster,Gryffindor,"15"" Elder Thestral tail hair core",Phoenix,2,2.0,22,1.0,Dumbledore's Army | Order of the Phoenix | Hog...,Considered by many to be one of the most power...,1,Headmaster,NaN,NaN,school head,NaN,NaN
108,5,Rubeus Hagrid,1,Keeper of Keys and Grounds | Professor of Care...,Gryffindor,"16"" Oak unknown core",NaN,1,1.0,2,1.0,Albus Dumbledore | Order of the Phoenix | Hogw...,Resistant to stunning spells| above average st...,2,Keeper of Keys and Grounds,Professor of Care of Magical Creatures,NaN,hogwarts service,professor,NaN


Respecte als personatges que tenen més d'una feina, mirarem quina d'aquestes està més repetida i li assignarem aquesta.

In [1776]:
# Creem un diccionari amb els valors de repetició de cada 'Job_1'
dict = df_job_1['Job_1_Agrupat'].value_counts().to_dict()

# Funció per assignar la feina que és repeteix més vegades
def feina_frequent(fila):
    jobs = [fila['Job_1_Agrupat'], fila.get('Job_2_Agrupat'), fila.get('Job_3_Agrupat')]
    jobs = [job for job in jobs if job]  # Eliminem les feines nul·les
    feina_mes_frequent = max(jobs, key=lambda job: dict.get(job, 0))
    return feina_mes_frequent

# Assignem la feina als personatges que només en tenen 1
df_job_1['Job_Final'] = df_job_1['Job_1_Agrupat']

# Assignem la feina més freqüent als personatges amb 2 o 3 feines
df_job_2['Job_Final'] = df_job_2.apply(feina_frequent, axis=1)
df_job_3['Job_Final'] = df_job_3.apply(feina_frequent, axis=1)

# Concatenem tots els datasets (afegim també els personatges que no tenen cap feina)
df_tot = pd.concat([df_sense_job, df_job_1, df_job_2, df_job_3], ignore_index=True, sort=False)

# Ordenem el dataset respecte Id
df_tot = df_tot.sort_values(by='Id')

# Assignem els valors de Job_Final a Job
df_tot['Job'] = df_tot['Job_Final']

# Eliminem les columnes que ja no necessitem
df = df_tot.drop(columns=['Job_1', 'Job_2', 'Job_3', 'Job_1_Agrupat', 'Job_2_Agrupat', 'Job_3_Agrupat', 'Job_Final', 'Job_Count'])

df.head()

C:\Users\nerea\AppData\Local\Temp\ipykernel_14672\2209735659.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_job_1['Job_Final'] = df_job_1['Job_1_Agrupat']
C:\Users\nerea\AppData\Local\Temp\ipykernel_14672\2209735659.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_job_2['Job_Final'] = df_job_2.apply(feina_frequent, axis=1)
C:\Users\nerea\AppData\Local\Temp\ipykernel_14672\2209735659.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

,Id,Name,Gender,Job,House,Wand,Patronus,Species,Blood status,Hair colour,Eye colour,Loyalty,Skills
5,1,Harry James Potter,1,student,Gryffindor,"11"" Holly phoenix feather",Stag,2,2.0,2,1.0,Albus Dumbledore | Dumbledore's Army | Order o...,Parseltongue| Defence Against the Dark Arts | ...
6,2,Ronald Bilius Weasley,1,student,Gryffindor,"12"" Ash unicorn tail hair",Jack Russell terrier,2,2.0,16,1.0,Dumbledore's Army | Order of the Phoenix | Hog...,Wizard chess | Quidditch goalkeeping
7,3,Hermione Jean Granger,0,student,Gryffindor,"10¾"" vine wood dragon heartstring",Otter,2,2.0,5,0.0,Dumbledore's Army | Order of the Phoenix | Hog...,Almost everything
8,4,Albus Percival Wulfric Brian Dumbledore,1,school head,Gryffindor,"15"" Elder Thestral tail hair core",Phoenix,2,2.0,22,1.0,Dumbledore's Army | Order of the Phoenix | Hog...,Considered by many to be one of the most power...
108,5,Rubeus Hagrid,1,professor,Gryffindor,"16"" Oak unknown core",NaN,1,1.0,2,1.0,Albus Dumbledore | Order of the Phoenix | Hogw...,Resistant to stunning spells| above average st...


In [1777]:
# Eliminem espai del final
df['Skills'] = df['Skills'].str.strip()

# Eliminem els parèntesis explicatius de la columna 'Skills'
df['Skills'] = df['Skills'].str.replace(r'\s*\(.*?\)', '', regex=True)

# Verifiquem els resultats
df['Skills'].unique()

# Primer, ignorem les files amb NaNs a la columna 'Skills' al contar les feines
df['Skills_Count'] = df['Skills'].apply(
    lambda x: len(x.split('|')) if isinstance(x, str) and '|' in x else 
            (len(x.split(',')) if isinstance(x, str) and ',' in x else 
            1 if isinstance(x, str) else 0)
)

# Filtrem les files on 'Skills_Count' es més gran que 1
multiple_jobs = df[df['Skills_Count'] > 1]

# Imprimim els personatges que tenen més d'una feina
print("Personatges amb més d'una feina:")
print(multiple_jobs[['Skills', 'Skills_Count']])

print("\nEl màxim de Skills que te algún dels personatges és de",max(multiple_jobs['Skills_Count']))


Personatges amb més d'una feina:
                                                Skills  Skills_Count
5    Parseltongue| Defence Against the Dark Arts | ...             3
6                 Wizard chess | Quidditch goalkeeping             2
108  Resistant to stunning spells| above average st...             3
12                               Chaser| Bat-Bogey hex             2
16                                    Animagus| Seeker             2
17                   Animagus| nonverbal magic| Charms             3
116         Intelligent| hard-working| a model student             3
20   Highly skilled curse-breaker for Gringotts Wiz...             3
21   Highly skilled dragon trainer| one-time Gryffi...             2
22                 Quidditch commentary | broadcasting             2
23   Keeper|  Captain of the Gryffindor Quidditch team             2
32                     Household spells| healing magic             2
34   Learned in the theory of Defensive Magic| less...             2
4

In [1778]:
# Separem les feines en 3 columnes
import re

# Dividimos la columna `Skills` usando los separadores `|` y `,`
skills_separades = df['Skills'].apply(
    lambda x: re.split(r'[|,]', x) if pd.notna(x) else []
).apply(pd.Series)

# Canviem el nom de les columnes
skills_separades.columns = [f'Skill_{i+1}' for i in range(skills_separades.shape[1])]

# Afegim les columnes al df
df = pd.concat([df, skills_separades], axis=1)

df.head(100)

,Id,Name,Gender,Job,House,Wand,Patronus,Species,Blood status,Hair colour,Eye colour,Loyalty,Skills,Skills_Count,Skill_1,Skill_2,Skill_3,Skill_4,Skill_5,Skill_6
5,1,Harry James Potter,1,student,Gryffindor,"11"" Holly phoenix feather",Stag,2,2.0,2,1.0,Albus Dumbledore | Dumbledore's Army | Order o...,Parseltongue| Defence Against the Dark Arts | ...,3,Parseltongue,Defence Against the Dark Arts,Seeker,NaN,NaN,NaN
6,2,Ronald Bilius Weasley,1,student,Gryffindor,"12"" Ash unicorn tail hair",Jack Russell terrier,2,2.0,16,1.0,Dumbledore's Army | Order of the Phoenix | Hog...,Wizard chess | Quidditch goalkeeping,2,Wizard chess,Quidditch goalkeeping,NaN,NaN,NaN,NaN
7,3,Hermione Jean Granger,0,student,Gryffindor,"10¾"" vine wood dragon heartstring",Otter,2,2.0,5,0.0,Dumbledore's Army | Order of the Phoenix | Hog...,Almost everything,1,Almost everything,NaN,NaN,NaN,NaN,NaN
8,4,Albus Percival Wulfric Brian Dumbledore,1,school head,Gryffindor,"15"" Elder Thestral tail hair core",Phoenix,2,2.0,22,1.0,Dumbledore's Army | Order of the Phoenix | Hog...,Considered by many to be one of the most power...,1,Considered by many to be one of the most power...,NaN,NaN,NaN,NaN,NaN
108,5,Rubeus Hagrid,1,professor,Gryffindor,"16"" Oak unknown core",NaN,1,1.0,2,1.0,Albus Dumbledore | Order of the Phoenix | Hogw...,Resistant to stunning spells| above average st...,3,Resistant to stunning spells,above average strength,crossbowmanship,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,102,Amos Diggory,1,magical government,Hufflepuff,Unknown,Non-corporeal,2,2.0,5,1.0,Minister of Magic,Works for the Department for the Regulation an...,1,Works for the Department for the Regulation an...,NaN,NaN,NaN,NaN,NaN
92,105,Fleur Isabelle Delacour,0,gringotts,Beauxbatons Academy of Magic,"9½"", Rosewood, veela hair",Non-corporeal,2,2.0,21,0.0,Order of the Phoenix,part Veela and therefore possesses some of the...,1,part Veela and therefore possesses some of the...,NaN,NaN,NaN,NaN,NaN
93,106,Aberforth Dumbledore,1,barman,Gryffindor,Unknown,Goat,2,2.0,10,1.0,Order of the Phoenix,"Bartending, caring for Ariana",2,Bartending,caring for Ariana,NaN,NaN,NaN,NaN
94,110,Marlene McKinnon,0,auror,Gryffindor,Unknown,Unknown,2,2.0,16,0.0,Original Order of the Phoenix,Auror,1,Auror,NaN,NaN,NaN,NaN,NaN


In [1779]:
# Creem un diccionari de mapeig per agrupar feines similars
skills_similars = {
    'prefect': ['prefect', 'head boy'],
    'Duel': ['skilled duellist', 'duelling', 'duellist', 'combat'],
    'divination': ['divination', 'seer'],
    'Charms': ['charms', 'hex', 'spells', 'transfiguration', 'defensive magic', 'occlumens'],
    'Mail': ['propaganda'],
    'Heal': ['curing', 'medical', 'healing'],
    'Potions': ['potioneer', 'potions'],
    'auror': ['auror'],
    'magical creatures': ['magical creatures', 'giants','nargles', 'Magizoology'],
    'parsel': ['parsel', 'parselmouths'],
    'great wizard': ['powerful', 'great wizard'],
    'dark arts': ['curse', 'dark', 'voldemort', 'punishment', 'betraying', 'undercover', 'manipulation', 'heir of slytherin', 'inferi'],
    'veela': ['veela'],
    'wand': ['wandcraft'],
    'Quidditch' : ['chaser', 'seeker', 'beater', 'keeper', 'quidditch', 'captain', 'flying'],
    'wisdom': ['intelligence', 'everything', 'wisdom', 'model student', 'intelligent', 'skilled', 'revising'],
    'dancer': ['dancer'],
    'no skills': ['less', 'above average'],
    'professor': ['teaching', 'professor'],
    'other' : ['peeves', 'love potion', 'caring'],
    'order': ['order']    
}

def agrupar_skill(skill):
    # Ens assegurem que les paraules estiguin en minuscules i sense espais addicionals
    skill = skill.strip().lower()
    
    for categoria, similares in skills_similars.items():
        # Comparem la skill amb les paraules de la llista
        llista = [item.strip().lower() for item in similares]
        paraules_dividides = []
        for paraula in llista:
            paraula_separada = paraula.split(' ')
            paraules_dividides.append(' '.join(paraula_separada))
        for i in paraules_dividides:
            if i in skill:
                return categoria.lower()
    return skill

# Filtrem els personatges que tenen 1, 2 o 3 feines
df_skill_1 = df[df['Skills_Count'] == 1]
df_skill_2 = df[df['Skills_Count'] == 2]
df_skill_3 = df[df['Skills_Count'] == 3]
df_skill_4 = df[df['Skills_Count'] == 4]
df_skill_5 = df[df['Skills_Count'] == 5]
df_skill_6 = df[df['Skills_Count'] == 6]

# Ens guardem també els personatges que no tenen cap feina
df_sense_skill = df[df['Skills_Count'] == 0]

# Apliquem la funció per agrupar les feines més similars de la columna 'Job_1'
df_skill_1['Skill_1_Agrupat'] = df_skill_1['Skill_1'].apply(agrupar_skill)

# Apliquem la funció per agrupar les feines més similars de la columna 'Job_2'
df_skill_2['Skill_1_Agrupat'] = df_skill_2['Skill_1'].apply(agrupar_skill)
df_skill_2['Skill_2_Agrupat'] = df_skill_2['Skill_2'].apply(agrupar_skill)

# Apliquem la funció per agrupar les feines més similars de la columna 'Job_3'
df_skill_3['Skill_1_Agrupat'] = df_skill_3['Skill_1'].apply(agrupar_skill)
df_skill_3['Skill_2_Agrupat'] = df_skill_3['Skill_2'].apply(agrupar_skill)
df_skill_3['Skill_3_Agrupat'] = df_skill_3['Skill_3'].apply(agrupar_skill)

df_skill_4['Skill_1_Agrupat'] = df_skill_4['Skill_1'].apply(agrupar_skill)
df_skill_4['Skill_2_Agrupat'] = df_skill_4['Skill_2'].apply(agrupar_skill)
df_skill_4['Skill_3_Agrupat'] = df_skill_4['Skill_3'].apply(agrupar_skill)
df_skill_4['Skill_4_Agrupat'] = df_skill_4['Skill_4'].apply(agrupar_skill)

df_skill_5['Skill_1_Agrupat'] = df_skill_5['Skill_1'].apply(agrupar_skill)
df_skill_5['Skill_2_Agrupat'] = df_skill_5['Skill_2'].apply(agrupar_skill)
df_skill_5['Skill_3_Agrupat'] = df_skill_5['Skill_3'].apply(agrupar_skill)
df_skill_5['Skill_4_Agrupat'] = df_skill_5['Skill_4'].apply(agrupar_skill)
df_skill_5['Skill_5_Agrupat'] = df_skill_5['Skill_5'].apply(agrupar_skill)

df_skill_6['Skill_1_Agrupat'] = df_skill_6['Skill_1'].apply(agrupar_skill)
df_skill_6['Skill_2_Agrupat'] = df_skill_6['Skill_2'].apply(agrupar_skill)
df_skill_6['Skill_3_Agrupat'] = df_skill_6['Skill_3'].apply(agrupar_skill)
df_skill_6['Skill_4_Agrupat'] = df_skill_6['Skill_4'].apply(agrupar_skill)
df_skill_6['Skill_5_Agrupat'] = df_skill_6['Skill_5'].apply(agrupar_skill)
df_skill_6['Skill_6_Agrupat'] = df_skill_6['Skill_6'].apply(agrupar_skill)

# Contem quantes vegades és repeteixen les feines de la columna 'Job_1'
skills_repetides_skill_1 = df_skill_1['Skill_1_Agrupat'].value_counts()

# Imprimim les feines i quantes vegeades és repeteixen
print("Feines repetides Job_1:\n")
print(skills_repetides_skill_1)

# Concatenem tots els datasets
df_tot = pd.concat([df_sense_skill, df_skill_1, df_skill_2, df_skill_3, df_skill_4, df_skill_5, df_skill_6], ignore_index=True, sort=False)

# Ordenem el dataset respecte Id
df_tot = df_tot.sort_values(by='Id')

df_tot.head(108)

C:\Users\nerea\AppData\Local\Temp\ipykernel_14672\3865827766.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_skill_1['Skill_1_Agrupat'] = df_skill_1['Skill_1'].apply(agrupar_skill)


Feines repetides Job_1:

Skill_1_Agrupat
quidditch             16
duel                   9
charms                 8
dark arts              7
potions                5
magical creatures      4
prefect                4
auror                  4
herbology              3
divination             3
great wizard           2
other                  2
veela                  2
animagus               2
heal                   2
dancer                 1
mail                   1
metamorphmagus         1
wisdom                 1
parsel                 1
wand                   1
muggle world works     1
photography            1
pyrotechnics           1
astronomy              1
Name: count, dtype: int64


C:\Users\nerea\AppData\Local\Temp\ipykernel_14672\3865827766.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_skill_2['Skill_1_Agrupat'] = df_skill_2['Skill_1'].apply(agrupar_skill)
C:\Users\nerea\AppData\Local\Temp\ipykernel_14672\3865827766.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_skill_2['Skill_2_Agrupat'] = df_skill_2['Skill_2'].apply(agrupar_skill)
C:\Users\nerea\AppData\Local\Temp\ipykernel_14672\3865827766.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

,Id,Name,Gender,Job,House,Wand,Patronus,Species,Blood status,Hair colour,...,Skill_3,Skill_4,Skill_5,Skill_6,Skill_1_Agrupat,Skill_2_Agrupat,Skill_3_Agrupat,Skill_4_Agrupat,Skill_5_Agrupat,Skill_6_Agrupat
109,1,Harry James Potter,1,student,Gryffindor,"11"" Holly phoenix feather",Stag,2,2.0,2,...,Seeker,NaN,NaN,NaN,parsel,dark arts,quidditch,NaN,NaN,NaN
89,2,Ronald Bilius Weasley,1,student,Gryffindor,"12"" Ash unicorn tail hair",Jack Russell terrier,2,2.0,16,...,NaN,NaN,NaN,NaN,wizard chess,quidditch,NaN,NaN,NaN,NaN
6,3,Hermione Jean Granger,0,student,Gryffindor,"10¾"" vine wood dragon heartstring",Otter,2,2.0,5,...,NaN,NaN,NaN,NaN,wisdom,NaN,NaN,NaN,NaN,NaN
7,4,Albus Percival Wulfric Brian Dumbledore,1,school head,Gryffindor,"15"" Elder Thestral tail hair core",Phoenix,2,2.0,22,...,NaN,NaN,NaN,NaN,great wizard,NaN,NaN,NaN,NaN,NaN
110,5,Rubeus Hagrid,1,professor,Gryffindor,"16"" Oak unknown core",NaN,1,1.0,2,...,crossbowmanship,NaN,NaN,NaN,charms,no skills,crossbowmanship,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,117,Madame Olympe Maxime,0,school head,Beauxbatons Academy of Magic,Unknown,Unknown,1,1.0,5,...,NaN,NaN,NaN,NaN,dancer,NaN,NaN,NaN,NaN,NaN
79,118,Gabrielle Delacour,0,student,Beauxbatons Academy of Magic,Unknown,Unknown,2,2.0,21,...,NaN,NaN,NaN,NaN,veela,NaN,NaN,NaN,NaN,NaN
80,119,Viktor Krum,1,quidditch,Durmstrang Institute,"10¼"", Hornbeam, dragon heartstring",Unknown,2,2.0,8,...,NaN,NaN,NaN,NaN,quidditch,NaN,NaN,NaN,NaN,NaN
5,124,Dennis Creevey,1,student,Gryffindor,Unknown,Unknown,2,2.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1780]:
# Creem un diccionari amb els valors de repetició de cada 'Job_1'
dict = df_skill_1['Skill_1_Agrupat'].value_counts().to_dict()

# Funció per assignar la feina que és repeteix més vegades
def skill_frequent(fila):
    skills = [fila['Skill_1_Agrupat'], fila.get('Skill_2_Agrupat'), fila.get('Skill_3_Agrupat'), fila.get('Skill_4_Agrupat'), fila.get('Skill_5_Agrupat'), fila.get('Skill_6_Agrupat')]
    skills = [skill for skill in skills if skill]  # Eliminem les feines nul·les
    skill_mes_frequent = max(skills, key=lambda skill: dict.get(skill, 0))
    return skill_mes_frequent

# Assignem la feina als personatges que només en tenen 1
df_skill_1['Skill_Final'] = df_skill_1['Skill_1_Agrupat']

# Assignem la feina més freqüent als personatges amb 2 o 3 feines
df_skill_2['Skill_Final'] = df_skill_2.apply(skill_frequent, axis=1)
df_skill_3['Skill_Final'] = df_skill_3.apply(skill_frequent, axis=1)
df_skill_4['Skill_Final'] = df_skill_4.apply(skill_frequent, axis=1)
df_skill_5['Skill_Final'] = df_skill_5.apply(skill_frequent, axis=1)
df_skill_6['Skill_Final'] = df_skill_6.apply(skill_frequent, axis=1)

# Concatenem tots els datasets (afegim també els personatges que no tenen cap feina)
df_tot = pd.concat([df_sense_skill, df_skill_1, df_skill_2, df_skill_3, df_skill_4, df_skill_5, df_skill_6], ignore_index=True, sort=False)

# Ordenem el dataset respecte Id
df_tot = df_tot.sort_values(by='Id')

# Assignem els valors de Job_Final a Job
df_tot['Skills'] = df_tot['Skill_Final']

# Eliminem les columnes que ja no necessitem
df = df_tot.drop(columns=['Skill_1', 'Skill_2', 'Skill_3', 'Skill_4', 'Skill_5', 'Skill_6', 'Skill_1_Agrupat', 'Skill_2_Agrupat', 'Skill_3_Agrupat', 'Skill_4_Agrupat', 'Skill_5_Agrupat', 'Skill_6_Agrupat', 'Skill_Final', 'Skills_Count'])

df.head()

C:\Users\nerea\AppData\Local\Temp\ipykernel_14672\3374169850.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_skill_1['Skill_Final'] = df_skill_1['Skill_1_Agrupat']
C:\Users\nerea\AppData\Local\Temp\ipykernel_14672\3374169850.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_skill_2['Skill_Final'] = df_skill_2.apply(skill_frequent, axis=1)
C:\Users\nerea\AppData\Local\Temp\ipykernel_14672\3374169850.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

,Id,Name,Gender,Job,House,Wand,Patronus,Species,Blood status,Hair colour,Eye colour,Loyalty,Skills
109,1,Harry James Potter,1,student,Gryffindor,"11"" Holly phoenix feather",Stag,2,2.0,2,1.0,Albus Dumbledore | Dumbledore's Army | Order o...,quidditch
89,2,Ronald Bilius Weasley,1,student,Gryffindor,"12"" Ash unicorn tail hair",Jack Russell terrier,2,2.0,16,1.0,Dumbledore's Army | Order of the Phoenix | Hog...,quidditch
6,3,Hermione Jean Granger,0,student,Gryffindor,"10¾"" vine wood dragon heartstring",Otter,2,2.0,5,0.0,Dumbledore's Army | Order of the Phoenix | Hog...,wisdom
7,4,Albus Percival Wulfric Brian Dumbledore,1,school head,Gryffindor,"15"" Elder Thestral tail hair core",Phoenix,2,2.0,22,1.0,Dumbledore's Army | Order of the Phoenix | Hog...,great wizard
110,5,Rubeus Hagrid,1,professor,Gryffindor,"16"" Oak unknown core",NaN,1,1.0,2,1.0,Albus Dumbledore | Order of the Phoenix | Hogw...,charms


In [1781]:
print("Columnes amb nans:\n")
nan_columns = df.columns[df.isna().sum()>0].tolist()
print (df.loc[:,nan_columns].isna().sum()/len(df)*100)

nan_columns = df.columns[df.isna().sum()>0].tolist() # columnes que falta 1 o més valors al datset
print('\nNÚMERO DE NANS PER COLUMNA:')
print (df.loc[:,nan_columns].isna().sum()) # número de nans per columna

Columnes amb nans:

Job         4.273504
Patronus    5.128205
Skills      5.128205
dtype: float64

NÚMERO DE NANS PER COLUMNA:
Job         5
Patronus    6
Skills      6
dtype: int64


Fem encoding de la característica **Job** i també fem la predicció dels Nans de la característica amb KNN:

**Normalitzem** les dades. En concret, els atributs ** També normalitzarem el target **.